In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv(r"/kaggle/input/bank-marketing/bank-full.csv")

In [ ]:
data.head()

TARGET VARIABLE DISTRIBUTION

In [ ]:
import matplotlib as plt
ax = data["y"].value_counts().plot(kind="bar", figsize = (6,6), fontsize = 11, color = "orange")
ax.set_title("Bank details:",size=20, pad=5)
ax.set_ylabel("Categorie counts",fontsize = 11)

DATA TRANSFORMATION

 Mapping all object variables to integer variables.

In [ ]:
"""
list_of_object_variables = data.dtypes[data.dtypes=="object"].index.tolist()
list_of_object_variables
"""

In [ ]:
"""#removing target variable from list of variables that needs to be changed
list_of_object_variables.remove("y")"""

In [ ]:
"""list_of_object_variables.remove("month")"""

In [ ]:

data.month = pd.to_datetime(data.month, format = "%b").dt.month

In [ ]:
data.head(10)

In [ ]:
"""for col in list_of_object_variables:
    data[col] = data[col].replace(data[col].unique().tolist(),list(range(0,data[col].nunique())))"""

In [ ]:
"""data.head(10)"""

In [ ]:
print(data.nunique(),"\n\n")
data.info()

In [ ]:
#list of variables with 2 categories:
bi_list = data.nunique()[data.nunique()==2].index.tolist()
bi_list

In [ ]:
for col in bi_list:
    data[col] = data[col].replace(data[col].unique().tolist(),[0,1])

In [ ]:
data.head(10)

Getting dummy data

In [ ]:
data = pd.get_dummies(data, drop_first=True)
data.head(10)

DATA SCALING

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Scaling columns that have values greater than 1

to_scale = [col for col in data.columns if data[col].max()>1]
mms= MinMaxScaler()
scaled = mms.fit_transform(data[to_scale])
scaled = pd.DataFrame(scaled,columns=to_scale)

#Replace the original columns with the scaled ones
for col in scaled:
    data[col] = scaled[col]

data.head(10)

In [ ]:
data.columns

**MODELING WITHOUT SMOTE**

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop("y", axis=1)
y = data.y

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

print(f'''% Positive class in train = {np.round(y_train.value_counts(normalize=True)[1]*100,2)}
% Positive class in test = {np.round(y_test.value_counts(normalize=True)[1]*100,2)}''')

In [ ]:
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, plot_confusion_matrix

#Train
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
prediction = model.predict(X_test)

#Evaluating performance
print(f'Accuracy = {accuracy_score(y_test, prediction):.2f}\nRecall = {recall_score(y_test, prediction):.2f}\n')
cm = plot_confusion_matrix(model,X_test,y_test)
"""plt.figure(cm,size=(6,6))
plt.title("CONFUSION MATRIX WITHOUT SMOTE", size=20)

sns.heatmap(cm, annot=True, cmap="Blues")
"""

Hence the model has high accuracy but cannot be used. 
Recall = number of True Positives/ Total number of positives, which is 39% here.
Meaning the classifier poorly classifies the true class.

**MACHINE LEARNING WITH SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)

X_sm, y_sm = sm.fit_resample(X,y)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print("\nBalance of positive and negative classes (%):")
y_sm.value_counts(normalize = True) *100


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm, test_size=0.25, random_state=42)
#Train
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
prediction = model.predict(X_test)

#Evaluating performance
print(f'Accuracy = {accuracy_score(y_test, prediction):.2f}\nRecall = {recall_score(y_test, prediction):.2f}\n')
cm = plot_confusion_matrix(model,X_test,y_test)



SMOTE increases the recall from 39% to 97%.